In [1]:
# Import packages
import pandas as pd
from sqlalchemy import create_engine
import json
import os
import sys
sys.path.append(os.path.dirname(os.path.realpath('__file__'))+'/utils')
from versions import version_0, version_1, version_2
from engine import con
pd.set_option('display.max_rows', 200)

In [2]:
con = create_engine(os.getenv('EDM_DATA'))

In [3]:
# Build funtion to get the domains associated with a specific field
def get_domain(field, domain_path):
    with open(domain_path, "r") as domain:
        domain = json.load(domain)
    records = [i for i in domain['fields'] if i['field'] == field]
    if len(records) == 0: 
        print(f'cannot find field: {field} in domain table')
    else: 
        return records[0]['domain']

In [4]:
# Example
proxcode_domain = get_domain('proxcode', 'pluto_domains.json')
print(proxcode_domain)

{'0': 'Not Available', '1': 'Detached', '2': 'Semi-Attached', '3': 'Attached'}


In [5]:
# Example
proxcode_domain.keys()

dict_keys(['0', '1', '2', '3'])

In [6]:
# Specify latest PLUTO version
pluto = version_0
# Specify previous PLUTO version or the one you want to compare to
plutoprev = version_1

### Value check for the thirteen fields with expected domain values

##### includes summary statistics of unique value comparisions for the two PLUTO version, and lists of values that do not agree w/ domain list

### Zoning districts

In [7]:
sql = f'''
    WITH newcount as (
    WITH zdall as (
    SELECT zonedist1 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist2 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist3 as zonedist FROM {pluto}
    UNION ALL
    SELECT zonedist4 as zonedist FROM {pluto})
    SELECT zonedist, COUNT(*) 
    FROM zdall
    GROUP BY zonedist),
    prevcount as (
    WITH zdall as (
    SELECT zonedist1 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist2 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist3 as zonedist FROM {plutoprev}
    UNION ALL
    SELECT zonedist4 as zonedist FROM {plutoprev})
    SELECT zonedist, COUNT(*) 
    FROM zdall
    GROUP BY zonedist)
    SELECT a.zonedist, b.zonedist as zonedistprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.zonedist = b.zonedist
    ORDER BY diff DESC;
'''

In [8]:
df = pd.read_sql(sql=sql, con=con)

In [9]:
df[df['diff'] != 0]

,zonedist,zonedistprev,countnew,countprev,diff
0,None,None,2559707.0,NaN,NaN
1,None,None,NaN,2567266.0,NaN
2,R3A,R3A,54903.0,54847.0,56.0
3,PARK,PARK,2836.0,2818.0,18.0
4,R3-2,R3-2,63448.0,63432.0,16.0
5,R3X,R3X,51710.0,51695.0,15.0
6,R3-1,R3-1,51502.0,51487.0,15.0
7,R4-1,R4-1,50693.0,50679.0,14.0
8,C4-2,C4-2,1684.0,1678.0,6.0
9,R2,R2,40163.0,40157.0,6.0


### Commercial overlay

In [10]:
sql = f'''
    WITH newcount as (
    WITH olall as (
    SELECT overlay1 as overlay FROM {pluto}
    UNION ALL
    SELECT overlay2 as overlay FROM {pluto}
    )
    SELECT overlay, COUNT(*) 
    FROM olall
    GROUP BY overlay),
    prevcount as (
    WITH olall as (
    SELECT overlay1 as overlay FROM {plutoprev}
    UNION ALL
    SELECT overlay2 as overlay FROM {plutoprev})
    SELECT overlay, COUNT(*) 
    FROM olall
    GROUP BY overlay)
    SELECT a.overlay, b.overlay as overlayprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.overlay = b.overlay
    ORDER BY a.overlay;
'''

In [11]:
df = pd.read_sql(sql=sql, con=con)

In [12]:
df

,overlay,overlayprev,countnew,countprev,diff
0,C1-1,C1-1,1591.0,1590.0,1.0
1,C1-2,C1-2,11052.0,11031.0,21.0
2,C1-3,C1-3,13796.0,13805.0,-9.0
3,C1-4,C1-4,8094.0,8093.0,1.0
4,C1-5,C1-5,2541.0,2541.0,0.0
5,C2-1,C2-1,1681.0,1682.0,-1.0
6,C2-2,C2-2,5995.0,6024.0,-29.0
7,C2-3,C2-3,11936.0,11946.0,-10.0
8,C2-4,C2-4,16731.0,16728.0,3.0
9,C2-5,C2-5,1525.0,1523.0,2.0


### Special purpose district

In [13]:
sql = f'''
    WITH newcount as (
    WITH sdall as (
    SELECT spdist1 as spdist FROM {pluto}
    UNION ALL
    SELECT spdist2 as spdist FROM {pluto}
    UNION ALL
    SELECT spdist3 as spdist FROM {pluto}
    )
    SELECT spdist, COUNT(*) 
    FROM sdall
    GROUP BY spdist),
    prevcount as (
     WITH sdall as (
    SELECT spdist1 as spdist FROM {plutoprev}
    UNION ALL
    SELECT spdist2 as spdist FROM {plutoprev}
    UNION ALL
    SELECT spdist3 as spdist FROM {plutoprev}
    )
    SELECT spdist, COUNT(*) 
    FROM sdall
    GROUP BY spdist)
    SELECT a.spdist, b.spdist as spdistprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.spdist = b.spdist
    ORDER BY diff DESC;
'''

In [14]:
df = pd.read_sql(sql=sql, con=con)

In [15]:
df[df['diff'] != 0]

,spdist,spdistprev,countnew,countprev,diff
0,None,None,NaN,2481371.0,NaN
1,None,None,2475728.0,NaN,NaN
2,SRD,SRD,43921.0,43902.0,19.0
3,DB,DB,935.0,933.0,2.0
4,NA-1,NA-1,2410.0,2408.0,2.0
5,HRW,HRW,36.0,34.0,2.0
6,IN,IN,202.0,201.0,1.0
74,DJ,DJ,1562.0,1563.0,-1.0
75,J,J,468.0,469.0,-1.0
76,MiD,MiD,1485.0,1486.0,-1.0


In [16]:
#Check spdist domain
unique_spdist = df.spdist.tolist()
spdist_domain = list(get_domain('spdist1', 'pluto_domains.json').keys())

In [17]:
# in domain not in pluto:
[i for i in spdist_domain if i not in unique_spdist]

['MX-3']

In [18]:
# in pluto not in domain:
[i for i in unique_spdist if i not in spdist_domain]

[None, None, 'MX-16/EC-5', 'BSC', 'MX-20', 'MX-16/EC-6', 'MX-18', 'SW']

### Limited height

In [19]:
sql = f'''
    WITH newcount as (
    SELECT ltdheight, count(*)
    FROM {pluto}
    GROUP BY ltdheight),
    prevcount as (
    SELECT ltdheight, count(*)
    FROM {plutoprev}
    GROUP BY ltdheight)
    SELECT a.ltdheight, b.ltdheight as ltdheightprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ltdheight = b.ltdheight
    ORDER BY a.ltdheight;
'''

In [20]:
df = pd.read_sql(sql=sql, con=con)

In [21]:
df

,ltdheight,ltdheightprev,countnew,countprev,diff
0,LH-1,LH-1,2239.0,2239.0,0.0
1,LH-1A,LH-1A,797.0,797.0,0.0
2,None,None,856136.0,NaN,NaN
3,None,None,NaN,858017.0,NaN


In [22]:
#Check ltdheight domain
unique_ltdheight = df.ltdheight.tolist()
ltdheight_domain = list(get_domain('ltdheight', 'pluto_domains.json').keys())

In [23]:
# in domain not in pluto:
[i for i in ltdheight_domain if i not in unique_ltdheight]

['LH-2', 'LH-3']

In [24]:
# in pluto not in domain:
[i for i in unique_ltdheight if i not in ltdheight_domain]

[None, None]

### Building class

In [25]:
sql = f'''
    WITH newcount as (
    SELECT bldgclass, count(*)
    FROM {pluto}
    GROUP BY bldgclass),
    prevcount as (
    SELECT bldgclass, count(*)
    FROM {plutoprev}
    GROUP BY bldgclass)
    SELECT a.bldgclass, b.bldgclass as bldgclassprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.bldgclass = b.bldgclass
    ORDER BY diff DESC;
'''

In [26]:
df = pd.read_sql(sql=sql, con=con)

In [27]:
df[df['diff'] != 0]

,bldgclass,bldgclassprev,countnew,countprev,diff
0,None,None,NaN,429.0,NaN
1,None,None,400.0,NaN,NaN
2,None,L2,NaN,1.0,NaN
3,None,L3,NaN,3.0,NaN
4,None,L8,NaN,12.0,NaN
5,None,L9,NaN,23.0,NaN
6,C1,C1,15707.0,15584.0,123.0
7,V1,V1,6838.0,6731.0,107.0
8,B2,B2,79511.0,79407.0,104.0
9,B3,B3,60421.0,60349.0,72.0


In [28]:
#Check bldgclass domain
unique_bldgclass = df.bldgclass.tolist()
bldgclass_domain = list(get_domain('bldgclass', 'pluto_domains.json').keys())

In [29]:
# in domain not in pluto:
[i for i in bldgclass_domain if i not in unique_bldgclass]

['A',
 'B',
 'C',
 'D',
 'E',
 'E3',
 'E4',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'L1',
 'L2',
 'L3',
 'L8',
 'L9',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'R7',
 'RT',
 'S',
 'T',
 'U',
 'U3',
 'V',
 'W',
 'Y',
 'Z',
 'Z6']

In [30]:
# in pluto not in domain:
[i for i in unique_bldgclass if i not in bldgclass_domain]

[None, None, None, None, None, None]

### Land use

In [31]:
sql = f'''
    WITH newcount as (
    SELECT landuse, count(*)
    FROM {pluto}
    GROUP BY landuse),
    prevcount as (
    SELECT landuse, count(*)
    FROM {plutoprev}
    GROUP BY landuse)
    SELECT a.landuse, b.landuse as landuseprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.landuse = b.landuse
    ORDER BY a.landuse;
'''

In [32]:
df = pd.read_sql(sql=sql, con=con)

In [33]:
df[df['diff'] != 0]

,landuse,landuseprev,countnew,countprev,diff
0,01,01,565559.0,565492.0,67.0
1,02,02,131406.0,131989.0,-583.0
2,03,03,12645.0,13690.0,-1045.0
3,04,04,55063.0,54964.0,99.0
4,05,05,21630.0,21609.0,21.0
5,06,06,9877.0,9954.0,-77.0
6,07,07,6341.0,6315.0,26.0
7,08,08,12167.0,12192.0,-25.0
8,09,09,4045.0,4034.0,11.0
9,10,10,10002.0,10260.0,-258.0


In [34]:
#Check landuse domain
unique_landuse = df.landuse.tolist()
landuse_domain = list(get_domain('landuse', 'pluto_domains.json').keys())

In [35]:
# in domain not in pluto:
[i for i in landuse_domain if i not in unique_landuse]

[]

In [36]:
# in pluto not in domain:
[i for i in unique_landuse if i not in landuse_domain]

[None, None]

### Owner type

In [37]:
sql = f'''
    WITH newcount as (
    SELECT ownertype, count(*)
    FROM {pluto}
    GROUP BY ownertype),
    prevcount as (
    SELECT ownertype, count(*)
    FROM {plutoprev}
    GROUP BY ownertype)
    SELECT a.ownertype, b.ownertype as ownertypeprev, a.count as countnew, b.count as countprev,  a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ownertype = b.ownertype
    ORDER BY a.ownertype;
'''

In [38]:
df = pd.read_sql(sql=sql, con=con)

In [39]:
df

,ownertype,ownertypeprev,countnew,countprev,diff
0,C,C,11654.0,11654.0,0.0
1,M,M,80.0,80.0,0.0
2,O,O,1373.0,1373.0,0.0
3,P,P,522.0,522.0,0.0
4,X,X,18244.0,19988.0,-1744.0
5,None,None,827299.0,NaN,NaN
6,None,None,NaN,827436.0,NaN


In [40]:
#Check ownertype domain
unique_ownertype = df.ownertype.tolist()
ownertype_domain = list(get_domain('ownertype', 'pluto_domains.json').keys())

In [41]:
# in domain not in pluto:
[i for i in ownertype_domain if i not in unique_ownertype]

['NaN']

In [42]:
# in pluto not in domain:
[i for i in unique_ownertype if i not in ownertype_domain]

[None, None]

### Area source

In [43]:
sql = f'''
    WITH newcount as (
    SELECT areasource, count(*)
    FROM {pluto}
    GROUP BY areasource),
    prevcount as (
    SELECT areasource, count(*)
    FROM {plutoprev}
    GROUP BY areasource)
    SELECT a.areasource, b.areasource as areasourceprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.areasource = b.areasource
    ORDER BY a.areasource;
'''

In [44]:
df = pd.read_sql(sql=sql, con=con)

In [45]:
df

,areasource,areasourceprev,countnew,countprev,diff
0,0,0,240.0,163.0,77.0
1,2,2,814949.0,816705.0,-1756.0
2,4,4,340.0,259.0,81.0
3,5,5,182.0,119.0,63.0
4,7,7,43061.0,43378.0,-317.0
5,None,None,400.0,NaN,NaN
6,None,None,NaN,429.0,NaN


In [46]:
#Check areasource domain
unique_areasource = df.areasource.tolist()
areasource_domain = list(get_domain('areasource', 'pluto_domains.json').keys())

In [47]:
# in domain not in pluto:
[i for i in areasource_domain if i not in unique_areasource]

[]

In [48]:
# in pluto not in domain:
[i for i in unique_areasource if i not in areasource_domain]

[None, None]

### Extension

In [49]:
sql = f'''
    WITH newcount as (
    SELECT ext, count(*)
    FROM {pluto}
    GROUP BY ext),
    prevcount as (
    SELECT ext, count(*)
    FROM {plutoprev}
    GROUP BY ext)
    SELECT a.ext, b.ext as extprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.ext = b.ext
    ORDER BY a.ext;
'''

In [50]:
df = pd.read_sql(sql=sql, con=con)

In [51]:
df

,ext,extprev,countnew,countprev,diff
0,E,E,48315.0,48333.0,-18.0
1,EG,EG,36674.0,36692.0,-18.0
2,G,G,264306.0,264441.0,-135.0
3,N,N,451337.0,451458.0,-121.0
4,None,None,58540.0,NaN,NaN
5,None,None,NaN,60129.0,NaN


In [52]:
#Check extension domain
unique_extension = df.ext.tolist()
extension_domain = list(get_domain('ext', 'pluto_domains.json').keys())

In [53]:
# in domain not in pluto:
[i for i in extension_domain if i not in unique_extension]

[]

In [54]:
# in pluto not in domain:
[i for i in unique_extension if i not in extension_domain]

['N', None, None]

### Proxy code

In [55]:
sql = f'''
    WITH newcount as (
    SELECT proxcode, count(*)
    FROM {pluto}
    GROUP BY proxcode),
    prevcount as (
    SELECT proxcode, count(*)
    FROM {plutoprev}
    GROUP BY proxcode)
    SELECT a.proxcode, b.proxcode as proxcodeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.proxcode = b.proxcode
    ORDER BY a.proxcode;
'''

In [56]:
df = pd.read_sql(sql=sql, con=con)

In [57]:
df

,proxcode,proxcodeprev,countnew,countprev,diff
0,0,0,143654.0,145130.0,-1476.0
1,1,1,313835.0,314015.0,-180.0
2,2,2,200381.0,200468.0,-87.0
3,3,3,200902.0,201011.0,-109.0
4,None,None,400.0,NaN,NaN
5,None,None,NaN,429.0,NaN


In [58]:
#Check proxcode domain
unique_proxcode = df.proxcode.tolist()
proxcode_domain = list(get_domain('proxcode', 'pluto_domains.json').keys())

In [59]:
# in domain not in pluto:
[i for i in proxcode_domain if i not in unique_proxcode]

[]

In [60]:
# in pluto not in domain:
[i for i in unique_proxcode if i not in proxcode_domain]

[None, None]

### Lot type

In [61]:
sql = f'''
    WITH newcount as (
    SELECT lottype, count(*)
    FROM {pluto}
    GROUP BY lottype),
    prevcount as (
    SELECT lottype, count(*)
    FROM {plutoprev}
    GROUP BY lottype)
    SELECT a.lottype, b.lottype as lottypeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.lottype = b.lottype
    ORDER BY a.lottype;
'''

In [62]:
df = pd.read_sql(sql=sql, con=con)

In [63]:
df

,lottype,lottypeprev,countnew,countprev,diff
0,0,0,8929.0,10419.0,-1490.0
1,1,1,1461.0,1456.0,5.0
2,2,2,1825.0,1826.0,-1.0
3,3,3,100646.0,100645.0,1.0
4,4,4,3409.0,3411.0,-2.0
5,5,5,737505.0,737857.0,-352.0
6,6,6,2222.0,2229.0,-7.0
7,7,7,63.0,63.0,0.0
8,8,8,2522.0,2528.0,-6.0
9,9,9,190.0,190.0,0.0


In [64]:
#Check lottype domain
unique_lottype = df.lottype.tolist()
lottype_domain = list(get_domain('lottype', 'pluto_domains.json').keys())

In [65]:
# in domain not in pluto:
[i for i in lottype_domain if i not in unique_lottype]

[]

In [66]:
# in pluto not in domain:
[i for i in unique_lottype if i not in lottype_domain]

[None, None]

### Basement code

In [67]:
sql = f'''
    WITH newcount as (
    SELECT bsmtcode, count(*)
    FROM {pluto}
    GROUP BY bsmtcode),
    prevcount as (
    SELECT bsmtcode, count(*)
    FROM {plutoprev}
    GROUP BY bsmtcode)
    SELECT a.bsmtcode, b.bsmtcode as bsmtcodeprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.bsmtcode = b.bsmtcode
    ORDER BY a.bsmtcode;
'''

In [68]:
df = pd.read_sql(sql=sql, con=con)

In [69]:
df

,bsmtcode,bsmtcodeprev,countnew,countprev,diff
0,0,0,74328.0,74405.0,-77.0
1,1,1,153621.0,153648.0,-27.0
2,2,2,496102.0,495983.0,119.0
3,3,3,1518.0,1511.0,7.0
4,4,4,4731.0,4723.0,8.0
5,5,5,128472.0,130354.0,-1882.0
6,None,None,400.0,NaN,NaN
7,None,None,NaN,429.0,NaN


In [70]:
#Check bsmtcode domain
unique_bsmtcode = df.bsmtcode.tolist()
bsmtcode_domain = list(get_domain('bsmtcode', 'pluto_domains.json').keys())

In [71]:
# in domain not in pluto:
[i for i in bsmtcode_domain if i not in unique_bsmtcode]

[]

In [72]:
# in pluto not in domain:
[i for i in unique_bsmtcode if i not in bsmtcode_domain]

[None, None]

### PLUTO map id

In [73]:
sql = f'''
    WITH newcount as (
    SELECT plutomapid, count(*)
    FROM {pluto}
    GROUP BY plutomapid),
    prevcount as (
    SELECT plutomapid, count(*)
    FROM {plutoprev}
    GROUP BY plutomapid)
    SELECT a.plutomapid, b.plutomapid as plutomapidprev, a.count as countnew, b.count as countprev, a.count - b.count as diff
    FROM newcount a
    FULL OUTER JOIN prevcount b
    ON a.plutomapid = b.plutomapid
    ORDER BY a.plutomapid;
'''

In [74]:
df = pd.read_sql(sql=sql, con=con)

In [75]:
df

,plutomapid,plutomapidprev,countnew,countprev,diff
0,1,1,856830,856853,-23
1,2,2,1731,3561,-1830
2,3,3,396,425,-29
3,4,4,211,210,1
4,5,5,4,4,0


In [76]:
#Check plutomapid domain
unique_plutomapid = df.plutomapid.tolist()
plutomapid_domain = list(get_domain('plutomapid', 'pluto_domains.json').keys())

In [77]:
# in domain not in pluto:
[i for i in plutomapid_domain if i not in unique_plutomapid]

[]

In [78]:
# in pluto not in domain:
[i for i in unique_plutomapid if i not in plutomapid_domain]

[]

#### done